## Problem 1: difference_fanout

>Given a list of numbers, for each number generate a list of the differences between it and $n$ (known as the **fanout** value) following numbers in the list. Return a list of all the lists generated for each number. For members in last part of the list that have less than $n$ following members, calculate as many differences as possible.

``` Python
# example behavior
>>> difference_fanout([3,2,4,6,1], 3)
[[1 ,-1 ,-3], [2, 4, -1], [2, -3], [-5], []]
```
For extra credits (and some extra fun!), try to write your function only using list comprehension. 

### Solution: difference_fanout
``` Python
def difference_fanout(l, fanout):
    """ Return a list of differences for 
        each value with its following terms
        
        Parameters
        ----------
        l: List[Union(int,float)]
            Input list
            
        fanout: int
            Number of neighbors to compute difference with
        
        Returns
        -------
        List[List[int]]
    """
    return [[neighbor - base for neighbor 
             in l[i+1:min(i+1+fanout,len(l))]] 
            for i,base in enumerate(l)]
```

This solution uses two list comprehensions, nested together. For readability, the outer list comprehension should probably be written as a for loop, initializing an empty list and iterating through the input list to append the generated list for each value. 

For each member of the list of index $i$, we compute differences with the slice 
```Python
l[i+1:min(i+1+fanout,len(l))]```
where the `min` function is essential for handling the last few values of the list. Stare at it for awhile until it makes sense! 

### Extension
Recall from [earlier](http://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/Functions.html#Functions-are-Objects) that functions are, under the hood, just objects with some special operations that allow you to "call" a function. This means that you can pass other functions as parameters into our function. It is especially powerful since it enables us to generalize the purposes of our functions. For example, we don't have to limit our function to just computing the **difference** between members and their following terms; we can apply **any** *binary operation*. Instead of finding the difference, we can calculate the sum or product or even concatenate two strings for a list of string. The possibility is limitless. 
Armed with this knowledge, we can generalize the code.
```Python
def apply_fanout(l, fanout, op):
    """ Return a list of outputs for each value 
        after applying a binary operation between 
        the value and its following terms
        
        Parameters
        ----------
        l: List[Object]
            Input list
        
        fanout: int
            Number of neighbors to apply the operation with
        
        op: Func(Object, Object) -> Object
            Any binary operation to be applied to members. First
            input should be the member, and the second input should
            be a term following the member.
        
        Returns
        -------
        List[List[Object]]
    """
    return [[op(base, neighbor) for neighbor 
             in l[i+1:min(i+1+fanout,len(l))]] 
            for i,base in enumerate(l)]
```
Now, we can rewrite `difference_fanout` simply as
``` Python
def difference_fanout(l, fanout):
    def compute_inv_diff(a,b):
        return a-b
    return apply_fanout(l, fanout, compute_inv_diff)
```
We can easily change `compute_inv_diff` to some other function for a totally different use. 

## Problem 2: within_margin_percentage
> An algorithm is required to test out what percentage of the parts the factory is producing fall within a safety margin of the design specifications. Given a list of values recording the metrics of the manufactured parts, a list of values representing the desired metrics required by the design, and a margin of error allowed by the design, compute what percentage of the values are within the safety margin (<=)

``` Python
# example behavior
>>> within_margin_percentage([10,5,8,3,2],[9.3,4.2,8.9,3,1.2],0.5)
0.8
```

Only 1 value falls out of the margin of error, as `1.2` is deviating more than `0.5` from `2`. 

### Solution: within_margin_percentage
``` Python
def within_margin_percentage(desired, actual, margin):
    """ Compute the percentage of values that fall within
        a margin of error of the desired values
        
        Parameters
        ----------
        desired: List[Union(int,float)]
            The desired metrics
        
        actual: List[Union(int,float)]
            The actual metrics
        
        margin: Union(int,float)
            The allowed margin of error
        
        Returns
        -------
        float
    """
    count = 0 # counter of how many are within margin
    total = len(desired)
    for i in range(total):
        # if within range, count+=1
        count = count+1 if abs(desired[i]-actual[i]) <= margin else count
    return count / total
```

## Problem 3: is_palindrome
> A palindrome is a string that reads the same from left to right and from right to left. Strings like `racecar` and `Live on time, emit no evil` are palindromes. Notice how only valid alphanumeric characters are accounted for and how palindromes are not case-sensitive. Given a string, return whether or not it is a palindrome. 

``` Python
# example behavior
>>> is_palindrome("Step on no pets!")
True
>>> is_palindrome("'Tis not a palindrome")
False
```

**Tips**: `str.isalnum()` returns whether or not a string has purely alphanumeric characters (it works for one-character strings too)
``` Python
>>> "I love Python".isalnum()
False
>>> "IlovePython".isalnum()
True
```

### Solution: is_palindrome
``` Python
def is_palindrome(s):
    """ Given a string, determine if it is a palindrome.
        Whitespaces and cases are all ignored. Only
        alphanumeric characters are taken into consideration
        
        Parameters
        ----------
        s: Str
            Input string
        
        Returns
        -------
        bool
    """
    filtered_str = "".join([c.lower() if c.isalnum() else "" for c in s])
    
    # the center element in an odd number of characters does not need
    # to be considered
    for i in range(len(filtered_str)//2):
        if filtered_str[i] != filtered_str[-(i+1)]:
            return False
    
    # passing the for loop means all the pairs match up
    return True
```

## Problem 4: funny_text
> You're trying to manipulate a block of text for some fun effects. Tally up the number of occurrences of each word in the block of text; then replace the least common word with the most common word, the second least common with the second most common, and so on. Lowercase everything, but be sure to retain any non-alphabetical characters. Two words are two continuous blocks of alphabetical letters separated by some non-alphabetical character in between.

``` Python
# example behavior
>>> funny_text('I’m selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can’t handle me at my worst, then you sure as hell don’t deserve me at my best.')
'best’can if, but don to hard times. best control of, best out am mistakes make don deserve insecure little a hell. impatient selfish as m’sure hell then deserve worst my, me as t you handle and’sure at then deserve worst i.'
```

**Tips**: look into `Collections.Counter` for tallying words. Read up on its documentation for useful methods. Also, 

### Solution: funny_text
``` Python
from collections import Counter
import re
def funny_text(s):
    """ Takes a string and tally number of occurrecnes each unique 
        word. Replace the most common word with the least, and the
        second most with the second least, and so on. String is
        lowercased, and all other characters are retained
        
        Parameters
        ----------
        s: Str
            The input text
        
        Returns
        -------
        Str
    """
    s = s.lower()
    # [^a-z]+ is a regular expression pattern that looks for any
    # one or multiple continuous characters that are not from a-z. 
    # We use it as a delimiter to split the string up into words
    cleaned = re.split("[^a-z]+", s)
    
    # clean up any leftover empty strings
    cleaned = [w for w in cleaned if w != ""]
    c = Counter(cleaned)
    
    # find mapping from most common to least common, etc
    most_commons = list(c.most_common())
    mapping = dict() 
    for i,v in enumerate(most_commons):
        mapping[v[0]] = most_commons[len(most_commons)-(i+1)][0]
    
    # replace words
    new = "" # new string to build up from
    current_i = 0 # keep track of where we are on s
    
    for w in cleaned:
        i = s.find(w, current_i)
        # if not found, quit
        if i == -1:
            break
        new += s[current_i:i] # add up the non-alphabetical chars
        new += mapping[w] # add the new word
        current_i = i+len(w) # index points to end of current word
    
    if current_i < len(s): # if s ends with non-alphabetical
        new += s[current_i:] # add the rest
    
    return new
```

There are a number of hacks and tricks used in this solution. `re.split` splits up a string into a list of strings that are separated by special delimiters in the original string. In this case, `re.split("[^a-z]+", s)` identifies any block of characters not within a-z and make a cut there. The result is a list of words, without these non-alphabetical characters. 

The construction of the output string is also tricky. We already created a mapping from old words to new words, and we now just have to apply the mapping to every word. `current_i` keeps track of the index we're at in the original string. Each iteration of the for loop we look for the next word, concatenating any non-alphabetical characters in between two words and adding the new word's corresponding replacement. 

## Problem 5: concat_to_str
> Sometimes it is very important to handle different object types of inputs differently in a function. Let's say you're appending a list of objects into one single string. If the object is an integer, convert it into a string by spelling out each digit in base-10 in this format:
`142` $\rightarrow$ `one-four-two`; `-12` $\rightarrow$ `neg-one-two`. If the object is a float, just append it's integer part (obtained by rounding down) the same way and the string `"and float"`:
`12.324` $\rightarrow$ `one-two and float`. If the object is a string, keep it as is. If the object is `None`, switch to a new line. If the object is any other type, return `None`. Each object's transcription in **one line** is separated by `" | "`, and the result should be one large string. 

``` Python
# example behavior
>>> s = concat_to_str([12,-14.23,"Some magic number", None,
>>>                    "Aha", 10.1, 1, 2, None, 5])
>>> print(s)
one-two | neg-one-four and float | Some magic number
Aha | one-zero and float | one | two
five
```

**Tips**: checkout the `isinstance` function introduced [here](http://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/Basic_Objects.html) for different type handling. Also, consider creating a helper function for the conversion from integer to our special-format string, since we have to do it twice. It's always good to extrapolate repeated tasks into functions. You'll also need to hard-code the conversion from each digit to its English spellout. 

### Solution: concat_to_str
``` Python
def _int_to_str(n):
    """ Takes an integer and formats it into a special string 
        e.g. 142 -> "one-four-two"
             -12 -> "neg-one-two"
    """
    mapping = {"0":"zero","1":"one","2":"two","3":"three",
               "4":"four","5":"five","6":"six","7":"seven",
               "8":"eight","9":"nine","-":"neg"}
    
    out = []
    for digit in list(str(n)):
        out.append(mapping[digit])
    
    return "-".join(out)
    
def concat_to_str(l):
    """ Takes a list of objects and special-handles each 
        to concatenate into a long string. 
        Object can be a str, int, float, or None
        
        Parameters
        ----------
        l: List[Union(str, int, float, None)]
            Input list
        
        Returns
        -------
        Union(str, None)
    """
    # output string
    out = ""
    
    # current line's list; will reset after 
    # reaching a Newline (None)
    current = []
    for item in l:
        if isinstance(item, int):
            current.append(_int_to_str(item))
        elif isinstance(item, float):
            # int(any_float) rounds the float down 
            # to get an integer
            current.append(_int_to_str(int(item)) + " and float")
        elif isinstance(item, str):
            current.append(item)
        elif item is None:
            # create a new line and append the current 
            # line's content
            out += " | ".join(current) + "\n"
            # reset
            current = []
        else:
            return None
    
    # no need to check if the last item in `l` is None
    # since if so `current` would be empty and nothing
    # would be appended
    out += " | ".join(current)
    
    return out
```